In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('sample_data/finestapi_tracking_db.csv')

# Print schema of dataframe

# id: database table row identifier
# updated: update time of the row in question
# ship: name of the ship
# imo: IMO number of the ship
# lat: latitude (dd.dddd)
# long: longitude (ddd.dddd)
# sog: Speed Over Ground
# cog: Course Over Ground
# hdg: Heading
# depPort: UNLOCODE of the port of departure: EETLL or FIHEL
# etdSchedule: scheduled departure time given by the shipping company
# etd: departure time estimated by FinEstAPI
# atd: actual departure time of the ship
# arrPort: UNLOCODE of the port of arrival: EETLL or FIHEL
# etaSchedule: scheduled arrival time
# eta: estimated arrival time
# ata: actual arrival time

schema = df.dtypes
print(schema)

id               int64
updated         object
ship            object
imo              int64
lat            float64
long           float64
sog            float64
cog              int64
hdg              int64
depPort         object
etdSchedule     object
etd             object
atd             object
arrPort         object
etaSchedule     object
eta             object
ata             object
dtype: object


1. Data cleaning:
+ Ensure consistent ship name (some names are capitalized)
+ Drop rows with missing fields.
+ Ensure consistent date format across all columns.

In [3]:
# Ensure consistent ship name (some names are capitalized)
df['ship'] = df['ship'].str.lower()

# Drop rows with missing fields
df = df.dropna()

# Ensure consistent date format across all columns
# Convert all date columns to a consistent format with hour, minute, and second
date_columns = ['etdSchedule', 'etd', 'atd', 'etaSchedule', 'eta', 'ata']  # Replace with the actual column names
date_format = '%d/%m/%Y %H:%M:%S' # Replace with the desired date format

df[date_columns] = df[date_columns].apply(lambda col: 
                                          pd.to_datetime(col, format='mixed', dayfirst=True))

# Print the DataFrame with consistent date format
print(df)
num_rows = df.shape[0]
print(num_rows)

            id           updated       ship      imo      lat     long  sog  \
238       4374  05/04/2018 21:24   megastar  9773064  59.4452  24.7717  3.9   
243       4379  05/04/2018 21:27       star  9364722  60.1477  24.9149  3.2   
495       4631  06/04/2018 06:27   megastar  9773064  60.1478  24.9146  0.5   
497       4633  06/04/2018 06:28       star  9364722  59.4444  24.7705  0.7   
722       4859  06/04/2018 09:27   megastar  9773064  59.4451  24.7713  2.9   
...        ...               ...        ...      ...      ...      ...  ...   
823216  911728  14/03/2019 22:26   megastar  9773064  59.4453  24.7717  3.3   
823220  911732  14/03/2019 22:28       star  9364722  60.1481  24.9150  0.9   
823458  911970  15/03/2019 06:08  finlandia  9214379  60.1487  24.9112  1.3   
823652  912164  15/03/2019 07:26       star  9364722  59.4451  24.7715  2.5   
823659  912171  15/03/2019 07:29   megastar  9773064  60.1478  24.9146  1.1   

        cog  hdg depPort         etdSchedule       

2. Generate a pandas dataframe with the following metrics:
+ List of available ferry names in the dataset.
+ Ferry with the highest number of trips each month.
+ Ferry that adheres the most to the departure time.
+ Ferry that adheres the most to the arrival time.

In [13]:
# List of available ferry names in the dataset
ferry_names = df['ship'].unique()
print(ferry_names)

df['month'] = df['atd'].dt.month

# Group by 'month' and 'name', count occurrences, and get maximum count per month
ferry_group = df.groupby(['month', 'ship']).size().reset_index(name='count')
ferry_group = ferry_group.groupby('month', as_index=False).apply(lambda x: x[x['count'] == x['count'].max()]).reset_index()
print(ferry_group[['month', 'ship', 'count']])

['megastar' 'star' 'europa' 'finlandia']
    month       ship  count
0       1   megastar    352
1       2   megastar    162
2       3   megastar     82
3       3       star     82
4       4       star    153
5       5   megastar    190
6       6   megastar    181
7       6       star    181
8       7   megastar    176
9       8   megastar    173
10      9   megastar    184
11     10  finlandia    165
12     11  finlandia    179
13     11       star    179
14     12  finlandia    165


In [22]:
# Calculate the difference between departure time and estimated departure time
df['depTimeDiff'] = (df['atd'] - df['etd']).dt.total_seconds()
df['depTimeDiffAbs'] = df['depTimeDiff'].abs()

df['arrTimeDiff'] = (df['ata'] - df['eta']).dt.total_seconds()
df['arrTimeDiffAbs'] = df['arrTimeDiff'].abs()

# Group by 'ship_name' and calculate the sum of 'depTimeDiff' and 'arrTimeDiff'
total_dep_time_diff = df.groupby('ship')['depTimeDiffAbs'].sum().reset_index(name='totalDepTimeDiff')
total_arr_time_diff = df.groupby('ship')['arrTimeDiffAbs'].sum().reset_index(name='totalArrTimeDiff')

# Create the new DataFrame with the desired columns
time_adhere = pd.merge(total_dep_time_diff, total_arr_time_diff, on='ship')

ship_with_min_dep_time_diff = time_adhere.loc[total_dep_time_diff['totalDepTimeDiff'].idxmin()]
ship_with_min_arr_time_diff = time_adhere.loc[total_arr_time_diff['totalArrTimeDiff'].idxmin()]

print(time_adhere)
print(ship_with_min_dep_time_diff)
print(ship_with_min_arr_time_diff)

        ship  totalDepTimeDiff  totalArrTimeDiff
0     europa          496784.0           52303.0
1  finlandia         2112454.0          140056.0
2   megastar         2960160.0          149449.0
3       star         2640267.0          134115.0
ship                  europa
totalDepTimeDiff    496784.0
totalArrTimeDiff     52303.0
Name: 0, dtype: object
ship                  europa
totalDepTimeDiff    496784.0
totalArrTimeDiff     52303.0
Name: 0, dtype: object


3. Create a code that generates the following plots:
+ Bar plot displaying the total number of trips for each company.
+ Plot the changes in the difference between the estimated departure time
(ETD) and the actual departure time (ATD) over time, while considering
outliers.
+ Plot the changes in the difference between the estimated arrival time (ETA)
and the actual arrival time (ATA) over time, while considering outliers. 
+ Generate a scatter plot showing the relationship between 'Sog' (Speed over
Ground) and the trip duration.
+ Create a scatter plot with a best-fit line to depict the correlation between
delay in departure (ATD - ETD) and delay in arrival (ATA - ETA). What
conclusions can be drawn about this relationship?

In [23]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

# Create a new DataFrame with the ship name and count of rows
ship_counts = pd.DataFrame(df['ship'].value_counts()).reset_index()

# Rename the columns
ship_counts.columns = ['ship', 'count']
print(ship_counts)

source = ColumnDataSource(ship_counts)
output_notebook()  # For displaying the plot in a Jupyter Notebook

# Create the figure
total_trip_fig = figure(x_range=ship_counts['ship'], height=400, title="Total trips",
           toolbar_location=None, tools="")

total_trip_fig.vbar(x='ship', top='count', width=0.9, source=source)
total_trip_fig.xgrid.grid_line_color = None
total_trip_fig.y_range.start = 0
total_trip_fig.xaxis.axis_label = "Ship Name"
total_trip_fig.yaxis.axis_label = "Count"
show(total_trip_fig)

        ship  count
0   megastar   2099
1       star   1918
2  finlandia   1914
3     europa    471


Loading BokehJS ...

In [21]:
# Calculate the Z-score for each data point
def z_scores_cal(df_column):
    return (df_column - df_column.mean()) / df_column.std()

def outliers_cal(data_frame, z_scores, threshold):
    return data_frame[abs(z_scores) > threshold]

z_scores_dep = z_scores_cal(df['depTimeDiff'])
z_scores_arr = z_scores_cal(df['arrTimeDiff'])

# Define a threshold for Z-scores (e.g., 3 or -3)
threshold = 3

# Find outliers by filtering data points with Z-scores beyond the threshold
outliers_dep = outliers_cal(df, z_scores_dep, threshold)
outliers_arr = outliers_cal(df, z_scores_arr, threshold)

# Create the figure
dep_time_diff_fig = figure(title="Time departure difference plot", height=400, width=800, tools="")

# Plot the data points as scatter points
dep_time_diff_fig.circle(df['etdSchedule'], df['depTimeDiff'], size=1, color="navy", alpha=0.5)

# Plot the outliers as scatter points with a different color or marker
dep_time_diff_fig.circle(outliers_dep['etdSchedule'], outliers_dep['depTimeDiff'], size=10, color="red", alpha=0.5)

# Customize the plot
dep_time_diff_fig.xaxis.axis_label = "Data Point Index"
dep_time_diff_fig.yaxis.axis_label = "Time Departure Difference (seconds)"

# Show the plot
show(dep_time_diff_fig)

In [24]:
# Create the figure
arr_time_diff_fig = figure(title="Time arrival difference plot", height=400, width=800, tools="")

# Plot the data points as scatter points
arr_time_diff_fig.circle(df['etaSchedule'], df['arrTimeDiff'], size=1, color="navy", alpha=0.5)

# Plot the outliers as scatter points with a different color or marker
arr_time_diff_fig.circle(outliers_arr['etaSchedule'], outliers_arr['arrTimeDiff'], size=10, color="red", alpha=0.5)

# Customize the plot
arr_time_diff_fig.xaxis.axis_label = "Data Point Index"
arr_time_diff_fig.yaxis.axis_label = "Time Arrival Difference (seconds)"

# Show the plot
show(arr_time_diff_fig)

In [26]:
# Time duration
df['timeDuration'] = (df['ata'] - df['atd']).dt.total_seconds()

# Create the figure
time_sog_fig = figure(title="Relationship between 'Sog' (Speed overGround) and the trip duration", height=400, width=2000, tools="")

# Plot the data points as scatter points
time_sog_fig.circle(df['sog'], df['timeDuration'], size=5, color="navy", alpha=0.5)

# Customize the plot
time_sog_fig.xaxis.axis_label = "Speed overGround"
time_sog_fig.yaxis.axis_label = "Time Duration (seconds)"

# Show the plot
show(time_sog_fig)

In [28]:
import numpy as np

# Create the figure
dep_arr_fig = figure(title="Departure Delay vs Arrival Delay", x_axis_label="Departure Delay", y_axis_label="Arrival Delay", height=400, width=1000)

# Plot the scatter points
dep_arr_fig.circle(x=df['depTimeDiff'], y=df['arrTimeDiff'], size=8, fill_color='blue', line_color='black')

# Calculate the best-fit line
slope, intercept = np.polyfit(df['depTimeDiff'], df['arrTimeDiff'], deg=1)
best_fit_line = slope * df['depTimeDiff'] + intercept

# Plot the best-fit line
dep_arr_fig.line(df['depTimeDiff'], best_fit_line, line_color='red', line_width=2)

# Show the plot
show(dep_arr_fig)


### Speed analysis in different ship categories

In [29]:
# Calculate average speed for each ship
ship_speeds = df.groupby('ship')['sog'].mean()

# Convert index (ship names) and speeds to lists
ships = ship_speeds.index.tolist()
average_speeds = ship_speeds.tolist()

# Create a ColumnDataSource for the data
source = ColumnDataSource(data=dict(ships=ships, average_speeds=average_speeds))

# Create the figure and plot the average speeds
ship_speed_fig = figure(x_range=ships, title='Average Speed by Ship', x_axis_label='Ship', y_axis_label='Average Speed (knots)',
           height=400, toolbar_location=None, tools='')

ship_speed_fig.vbar(x='ships', top='average_speeds', width=0.9, source=source, line_color='black', fill_color='blue')

# Rotate x-axis labels if needed
ship_speed_fig.xaxis.major_label_orientation = 45

# Show the plot
show(ship_speed_fig)

### Compare distribution of time delay in departure and arrival

In [30]:
from bokeh.layouts import gridplot

# Create the histogram for departure delays
p1 = figure(title="Departure Delays", x_axis_label="Delay (minutes)", y_axis_label="Frequency")
hist_departure, edges_departure = np.histogram(df['depTimeDiff'], bins=50)
p1.quad(top=hist_departure, bottom=0, left=edges_departure[:-1], right=edges_departure[1:], fill_color='blue', line_color='black')

# Create the histogram for arrival delays
p2 = figure(title="Arrival Delays", x_axis_label="Delay (minutes)", y_axis_label="Frequency")
hist_arrival, edges_arrival = np.histogram(df['arrTimeDiff'], bins=50)
p2.quad(top=hist_arrival, bottom=0, left=edges_arrival[:-1], right=edges_arrival[1:], fill_color='green', line_color='black')

# Create a grid layout for the plots
grid = gridplot([[p1, p2]])

# Show the plots
show(grid)

### Speed correlation with lat and lon

In [31]:
# Create a ColumnDataSource for the data
source = ColumnDataSource(data=df)

# Create the figure and plot the speeds
speed_fig = figure(title='Speed Analysis', x_axis_label='Longitude', y_axis_label='Latitude',
           tools='pan,wheel_zoom,box_select', toolbar_location='above')

# Plot the speeds with different colors and sizes
speed_fig.scatter(x='long', y='lat', size='sog',
          line_color='black', alpha=0.7, source=source)

# Customize other visual properties
speed_fig.title.align = 'center'
speed_fig.title.text_font_size = '16px'

# Show the plot
show(speed_fig)


### Departure and Arrival port analysis

In [32]:
# Calculate the frequency of ships visiting each departure port
departure_counts = df['depPort'].value_counts()

# Calculate the frequency of ships visiting each arrival port
arrival_counts = df['arrPort'].value_counts()

# Identify the most common departure port
most_common_departure = departure_counts.idxmax()

# Identify the most common arrival port
most_common_arrival = arrival_counts.idxmax()

# Calculate the average duration of ships' stays at each port
duration_by_port = df.groupby('depPort')[['eta', 'atd']].apply(lambda x: (x['eta'] - x['atd']).mean())

# Print the results
print("Frequency of ships visiting each departure port:")
print(departure_counts)
print("\nFrequency of ships visiting each arrival port:")
print(arrival_counts)
print("\nMost common departure port:", most_common_departure)
print("Most common arrival port:", most_common_arrival)
print("\nAverage duration of ships' stays at each port:")
print(duration_by_port)


Frequency of ships visiting each departure port:
depPort
FIHEL    3336
EETLL    3066
Name: count, dtype: int64

Frequency of ships visiting each arrival port:
arrPort
EETLL    3336
FIHEL    3066
Name: count, dtype: int64

Most common departure port: FIHEL
Most common arrival port: EETLL

Average duration of ships' stays at each port:
depPort
EETLL   0 days 02:18:59.957925636
FIHEL   0 days 02:13:54.085731414
dtype: timedelta64[ns]
